# ABS Quarterly Labour Account 6150

## Python set-up

In [1]:
# system imports
import textwrap

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt
import readabs as ra
from readabs import metacol

In [2]:
# local imports
from abs_helper import get_abs_data
from plotting import (
    recalibrate_series,
    plot_covid_recovery,
    line_plot,
    plot_growth_finalise,
    calc_growth,
)

In [3]:
# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
SHOW = False

## Get data from ABS

In [4]:
labour_cat = "6150.0.55.003"
abs_dict, meta, source, RECENT = get_abs_data(labour_cat)
plot_times = None, RECENT
plot_tags = "", "-recent"

In [5]:
# list tables
textwrap.wrap(", ".join(abs_dict.keys()), width=80)

['Industry summary table, 6150055003DO001, 6150055003DO002, 6150055003DO003,',
 '6150055003DO004, 6150055003DO005, 6150055003DO006, 6150055003DO007,',
 '6150055003DO008, 6150055003DO009, 6150055003DO010, 6150055003DO011,',
 '6150055003DO012, 6150055003DO013, 6150055003DO014, 6150055003DO015,',
 '6150055003DO016, 6150055003DO017, 6150055003DO018, 6150055003DO019,',
 '6150055003DO020, 6150055003DO021, 6150055003DO022']

## Plot

### Labour productivity - GDP per hour worked ...

In [6]:
# GDP ...
gdp_cat = "5206.0"
gdp_data, gdp_meta = ra.read_abs_series(
    cat=gdp_cat, 
    series_id=["A2304402X", "A2304404C"],  # GDP, GDP per capita, CVM, SeasAdj
    single_excel_only="5206001_Key_Aggregates"
)
gdp = gdp_data["A2304402X"] * 1_000_000  # GDP 

# Hours worked ...
labour_table = "6150055003DO001"
series_type = "Seasonally Adjusted"
did = "Volume; Labour Account hours actually worked in all jobs ;  Australia ;  Total all industries ;"
search = {labour_table: metacol.table, did: metacol.did, series_type: metacol.stype}
_table, hours_id, units = ra.find_id(meta, search)
hours = abs_dict[labour_table][hours_id] * 1_000  # hours worked

# calculate productivity and plot
productivity = gdp / hours
ax = productivity.dropna().plot(lw=2.5)
title = "Labour Productivity: GDP per Hour Worked"
lfooter = (
    f"Australia. {series_type.capitalize()}. GDP: Chain volume measures. "
    "Hours: Total actual hours worked. "
)
rfooter = f"ABS: {gdp_cat} {labour_cat}"
units = "$"

line_plot(
    productivity,
    dropna=True,
    title=title,
    ylabel=units,
    rfooter=rfooter,
    lfooter=lfooter,
    show=SHOW,
)

growth = calc_growth(productivity)
plot_growth_finalise(
    *growth,
    title=f"Growth in {title}",
    rfooter=rfooter,
    lfooter=lfooter,
    show=SHOW,
)

### Headline

In [7]:
series_type = "Seasonally Adjusted"
not_unit = "Percent"
table = "6150055003DO001"
data = abs_dict[table]

rows = meta[
    (meta[metacol.table] == table)
    & (meta[metacol.stype] == series_type)
    & (meta[metacol.unit] != not_unit)
]

for index, row in rows.iterrows():
    series_id, units, title = row[metacol.id], row[metacol.unit], row[metacol.did]
    plotable, units = ra.recalibrate(data[series_id], units)
    plotable.name = f"{series_type.capitalize()} series"

    plot_covid_recovery(
        plotable,
        title=(
            title.replace(" ;  Australia ;  Total all industries ;", "")
            .replace(" per Labour Account", "\nper Labour Account")
            .replace(" ; ", " - ")
            .replace("; ", ": ")
        ),
        tags="covid",
        ylabel=units,
        start_r="2009Q4",
        end_r="2019Q4",
        rfooter=f"{source} T{table}",
        lfooter="Australia. All industries. ",
        show=SHOW,
    )

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Jul 06 2024 21:01:30

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

readabs   : 0.0.6a10
pandas    : 2.2.2
matplotlib: 3.9.0

Watermark: 2.4.3



In [9]:
print("Finished")

Finished
